In [25]:
import datetime
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [26]:
tweets_all = pd.read_csv("./data/tweets_all.csv", encoding="latin1") 

In [27]:
tweets_all.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0


In [28]:
tweets_all.shape

(119850, 13)

In [29]:
london_tweets = tweets_all[tweets_all.LOCATION.str.contains("London")]

In [30]:
london_tweets = london_tweets[~(london_tweets.LOCATION == "London; England")]

In [31]:
london_tweets.shape

(97779, 13)

In [32]:
london_tweets.LOCATION = london_tweets.LOCATION.apply(lambda l: l.split(";")[0])
london_tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames,NaN,0
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington,NaN,0
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford,NaN,0
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford,NaN,0
7,550227602011856896,NaN,-0.104869,51.509530,-0.104869,51.509529,31/12/2014 09:50:57,151094999,HevnKISZ,@JJJ_Jeeppy >> # XD,Camberwell,,0


In [33]:
london_tweets = london_tweets[~(london_tweets.LOCATION == "Westerham")]

In [34]:
# Need to change
# * "New Addington; London" -> Croydon
# * "Stratford; London" -> Newham
# * "Romford; London" -> Havering
# * "Camberwell; London" -> Southwark
# * 'East Ham; London' -> New Ham
# 'Paddington; London' -> City of Westminster
# 'Kensington; London' -> Kensington and Chelsea
# 'Camden Town; London' -> Camden
# 'Walthamstow; London' -> Waltham Forest
# 'Poplar; London' -> Tower Hamlets
# 'Crayford; London' -> Bexley
# 'Ilford; London' -> Redbridge
# 'Eltham; London' -> Greenwich
# 'Richmond; London' -> Richmond upon Thames
# 'Hammersmith; London' -> Hammersmith and Fulham
# 'Barking; London' -> Barking and Dagenham
# 'Tottenham; London' -> Haringey
# 'Southall; London' -> Ealing
# 'Harefield; London' -> Hillingdon
# 'Downe; London' -> Bromley
# 'Biggin Hill; London' -> Bromley
# 'Hazlewood; London' -> Bromley
# 'Keston; London' -> Bromley
# 'Pratts Bottom; London' -> Bromley

In [35]:
# Full list of London Boroughs - including City of London (https://en.wikipedia.org/wiki/London_boroughs#List_of_boroughs)
# 33 in total

# * Barking and Dagenham
# * Barnet
# * Bexley
# * Brent
# * Bromley
# * Camden
# * City of London (not a London borough)
# * City of Westminster
# * Croydon
# * Ealing
# * Enfield
# * Greenwich
# * Hackney
# * Hammersmith and Fulham
# * Haringey
# * Harrow
# * Havering
# * Hillingdon
# * Hounslow
# * Islington
# * Kensington and Chelsea
# * Kingston upon Thames
# * Lambeth
# * Lewisham
# * Merton
# * Newham
# * Redbridge
# * Richmond upon Thames
# * Southwark
# * Sutton
# * Tower Hamlets
# * Waltham Forest
# * Wandsworth

In [39]:
locations_to_change = {
    "City of Westminster": "Westminster",
    "New Addington": "Croydon",
    "Stratford": "Newham",
    "Romford": "Havering",
    "Camberwell": "Southwark",
    "East Ham": "Newham",
    "Paddington": "City of Westminster",
    "Kensington": "Kensington and Chelsea",
    "Camden Town": "Camden",
    "Walthamstow": "Waltham Forest",
    "Poplar": "Tower Hamlets",
    "Crayford": "Bexley",
    "Ilford": "Redbridge",
    "Eltham": "Greenwich",
    "Richmond": "Richmond upon Thames",
    "Hammersmith": "Hammersmith and Fulham",
    "Barking": "Barking and Dagenham",
    "Tottenham": "Haringey",
    "Southall": "Ealing",
    "Harefield": "Hillingdon",
    "Downe": "Bromley",
    "Biggin Hill": "Bromley",
    "Hazlewood": "Bromley",
    "Keston": "Bromley",
    "Pratts Bottom": "Bromley"
}

In [40]:
def change_location_name(location):
    
    if location in locations_to_change.keys():
        return locations_to_change[location]
    else:
        return location

london_tweets.LOCATION = london_tweets.LOCATION.apply(change_location_name)

In [41]:
london_tweets.LOCATION.unique()

array(['Kingston upon Thames', 'Croydon', 'Newham', 'Havering',
       'Southwark', 'Hackney', 'Wandsworth', 'Westminster', 'Bexley',
       'Kensington and Chelsea', 'Islington', 'Hillingdon', 'Camden',
       'Barnet', 'Hounslow', 'City of London', 'Ealing', 'Lambeth',
       'Waltham Forest', 'Tower Hamlets', 'Harrow', 'Bromley',
       'Redbridge', 'Brent', 'Greenwich', 'Richmond upon Thames',
       'Enfield', 'Sutton', 'Lewisham', 'Hammersmith and Fulham',
       'Barking and Dagenham', 'Haringey', 'Merton'], dtype=object)

In [42]:
london_tweets.LOCATION.unique().shape

(33,)

In [43]:
l = london_tweets.LOCATION.unique().tolist()

In [44]:
len(l)

33

In [45]:
l.sort()
l

['Barking and Dagenham',
 'Barnet',
 'Bexley',
 'Brent',
 'Bromley',
 'Camden',
 'City of London',
 'Croydon',
 'Ealing',
 'Enfield',
 'Greenwich',
 'Hackney',
 'Hammersmith and Fulham',
 'Haringey',
 'Harrow',
 'Havering',
 'Hillingdon',
 'Hounslow',
 'Islington',
 'Kensington and Chelsea',
 'Kingston upon Thames',
 'Lambeth',
 'Lewisham',
 'Merton',
 'Newham',
 'Redbridge',
 'Richmond upon Thames',
 'Southwark',
 'Sutton',
 'Tower Hamlets',
 'Waltham Forest',
 'Wandsworth',
 'Westminster']

In [46]:
london_tweets.shape

(97778, 13)

In [47]:
london_tweets.to_csv("./data/london_tweets.csv", index=False)

In [48]:
london_tweets

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames,NaN,0
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,Croydon,NaN,0
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Newham,NaN,0
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Havering,NaN,0
7,550227602011856896,NaN,-0.104869,51.509530,-0.104869,51.509529,31/12/2014 09:50:57,151094999,HevnKISZ,@JJJ_Jeeppy >> # XD,Southwark,,0
8,550227616649994240,NaN,-0.076154,51.524464,-0.076154,51.524464,31/12/2014 09:51:01,19820211,NooshT,Coffee with Chloe; whilst watching Cat in the ...,Hackney,NaN,0
10,550227983462825984,NaN,0.167534,51.606304,0.167534,51.606304,31/12/2014 09:52:28,615529484,IAMLAURENGEORGE,@_LivGeorge @MagnaCarta_LDN sorry sold,Havering,NaN,0
11,550227983974162432,NaN,-0.113500,51.337700,-0.113500,51.337700,31/12/2014 09:52:28,428080660,MarieTasleJolly,Good Morning... @ Purley railway station http:...,Croydon,NaN,0
12,550228002525941760,NaN,-0.198773,51.462128,-0.198773,51.462128,31/12/2014 09:52:33,2745207615,DickyMalton,Last day of 2014; Richmond park @PictureOfLOND...,Wandsworth,NaN,0
13,550228343241838593,NaN,0.050057,51.526386,0.050057,51.526386,31/12/2014 09:53:54,348386293,peoffrey,A manager like Jurgen Klopp would be selling h...,Newham,NaN,0
